<a href="https://colab.research.google.com/github/NicolasRuiz889/Proyecto_final_sistemas_distribuidos_2025/blob/main/copia_de_2025_07_07_file_1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

\# MODELO DE PREDICCIÓN PARA OFFERED CALLS


\#\#\# 1. Importación de librerias


In [ ]:
pip install skforecast

In [ ]:
pip install feature_engine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.6/378.6 kB 4.6 MB/s eta 0:00:00


In [ ]:
pip install astral

In [ ]:
#==============================================================================
# ⚙️ Configuración inicial
# ==============================================================================
import warnings
warnings.filterwarnings('once')

# Colores para impresión
color = '\033[1m\033[38;5;208m'

# ==============================================================================
# 🧮 Librerías base
# ==============================================================================
import numpy as np
import pandas as pd
import re
from io import StringIO
import contextlib

# ==============================================================================
# 📊 Visualización
# ==============================================================================
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams.update({'font.size': 8})
from matplotlib.gridspec import GridSpec
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as poff
pio.templates.default = "seaborn"
poff.init_notebook_mode(connected=True)

# Statsmodels
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose

# skforecast visual tools
from skforecast.plot import (
    plot_residuals,
    calculate_lag_autocorrelation,
    set_dark_theme
)

# ==============================================================================
# 📦 Librerías de modelos y forecasting
# ==============================================================================
import sklearn
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import RFECV

import lightgbm
from lightgbm import LGBMRegressor

import shap

# skforecast (forecasting con regresores)
import skforecast
from skforecast.datasets import fetch_dataset
from skforecast.feature_selection import select_features
from skforecast.preprocessing import RollingFeatures
from skforecast.model_selection import (
    TimeSeriesFold,
    backtesting_forecaster,
    backtesting_sarimax,
    bayesian_search_forecaster,
    grid_search_sarimax
)
from skforecast.direct import ForecasterDirect
from skforecast.recursive import (
    ForecasterRecursive,
    ForecasterEquivalentDate,
    ForecasterSarimax
)
from skforecast.metrics import calculate_coverage
from skforecast.sarimax import Sarimax

# ==============================================================================
# 🕒 Ingeniería de características temporales
# ==============================================================================
from feature_engine.datetime import DatetimeFeatures
from feature_engine.creation import CyclicalFeatures
from feature_engine.timeseries.forecasting import WindowFeatures

# Astral para variables solares
from astral import LocationInfo
from astral.sun import sun

# ==============================================================================
# 🔎 Mostrar versiones principales
# ==============================================================================
print(f"{color}Version skforecast: {skforecast.__version__}")
print(f"{color}Version scikit-learn: {sklearn.__version__}")
print(f"{color}Version lightgbm: {lightgbm.__version__}")
print(f"{color}Version pandas: {pd.__version__}")
print(f"{color}Version numpy: {np.__version__}")

Version skforecast: 0.16.0
Version scikit-learn: 1.6.1
Version lightgbm: 4.5.0
Version pandas: 2.2.2
Version numpy: 2.0.2


<frozen importlib._bootstrap>:1047: ImportWarning:

_PyDriveImportHook.find_spec() not found; falling back to find_module()

<frozen importlib._bootstrap>:1047: ImportWarning:

_BokehImportHook.find_spec() not found; falling back to find_module()



In [ ]:

import pandas as pd

# 1) Leer todo el Excel
file_path = r'C:\Users\nicol\OneDrive\Escritorio\forecast\data (20).xlsx'
df_excel = pd.read_excel(file_path)

# 2) Guardar la primera aparición de “Total” en DAY - Year
year_total_df = df_excel[df_excel['DAY - Year'] == 'Total'].head(1).copy()

# 3) Encontrar el índice de esa fila
mask_year_total   = df_excel['DAY - Year'] == 'Total'
first_total_idx   = mask_year_total[mask_year_total].index[0]

# 4) Cortar todo lo que viene desde esa fila en adelante
df_trim = df_excel.iloc[:first_total_idx].copy()

# 5) A partir de aquí trabajas solo con df_trim
#    Por ejemplo, eliminar cualquier “Total” residual en Month/Day/Hour
df_clean = df_trim[
    (df_trim['DAY - Month'] != 'Total') &
    (df_trim['DAY - Day']   != 'Total') &
    (df_trim['Hour']        != 'Total')
].copy()

# 6) (Opcional) ya no haces dropna si no quieres eliminar NaNs legítimos
# 7) Normalizar Hour, construir fecha, etc.

# --- Al final tienes ---
print("Fila de Total en Year guardada:")
print(year_total_df)
print("\nData limpia lista para procesar:")
print(df_clean.head())

In [ ]:
df_clean

In [ ]:
# 2) Conteo de nulos por columna
null_counts = df_clean.isna().sum().reset_index()
null_counts.columns = ['Column', 'Null Count']
print("Null counts per column:")
print(null_counts.to_string(index=False))

# 3) Filas con al menos un nulo
nan_rows = df_clean[df_clean.isna().any(axis=1)]
print("\nRows containing any nulls:")
print(nan_rows)


In [ ]:
# 1) Normaliza Hour para quedarte solo con "HH:MM"
df_clean['Hour_str'] = df_clean['Hour'].astype(str).str[:5]

# 2) Construye la cadena y parsea
fecha_str = (
    df_clean['DAY - Year'].astype(int).astype(str)   + '-' +
    df_clean['DAY - Month']                          + '-' +
    df_clean['DAY - Day'].astype(int).astype(str)    + ' ' +
    df_clean['Hour_str']
)
df_clean['fecha'] = pd.to_datetime(fecha_str,
                                   format='%Y-%B-%d %H:%M',
                                   errors='raise')  # te avisará si aún hay algo mal

# 3) (Opcional) quita la columna auxiliar
df_clean.drop(columns=['Hour_str'], inplace=True)

In [ ]:
df_clean

In [ ]:
# 7. Guardar el resultado limpio
output_csv = r'C:\Users\nicol\OneDrive\Escritorio\forecast\output.csv'
df_clean.to_csv(output_csv, index=False)

print("CSV limpio guardado en:", output_csv)
print("\nFila de total mensual extraída:")
print(month_total_df)

In [ ]:
import pandas as pd

# --- 1. Carga y preparación de df1 ---
df1 = pd.read_csv('output.csv', parse_dates=['fecha'], index_col='fecha')

df1 = df1.rename(columns={
    'Offered':   'OFF Calls',
    'Handled':   'Handled Calls',
    'AHT':       'Real AHT',
    'TSF':       'Real TSF',
    'ABA %':     'Real ABA%',
    'Calls SL':  'Real ABA',
    'ASA':       'Real ASA'
})

# df1 ya estaba en float, pero nos aseguramos de que TSF y ABA% sean float
df1['Real TSF']  = df1['Real TSF'].astype(float)
df1['Real ABA%'] = df1['Real ABA%'].astype(float)

cols = ['OFF Calls','Handled Calls','Real AHT','Real TSF','Real ABA%','Real ASA','Real ABA']
df1 = df1[cols]

# --- 2. Carga y preparación de df2 ---
df2 = pd.read_csv('data_jam.csv')

meses = {
    'enero':'January','febrero':'February','marzo':'March','abril':'April',
    'mayo':'May','junio':'June','julio':'July','agosto':'August',
    'septiembre':'September','octubre':'October','noviembre':'November','diciembre':'December'
}

df2['mes_ing'] = df2['DATE_S - Mes'].str.lower().map(meses)
df2['fecha']   = pd.to_datetime(
    df2['DATE_S - Día'].astype(str) + ' ' +
    df2['mes_ing'] + ' ' +
    df2['DATE_S - Año'].astype(str),
    format='%d %B %Y',
    errors='coerce'
)

# Generar índice continuo cada 30 minutos
inicio = df2['fecha'].iloc[0]
df2['fecha'] = [inicio + pd.Timedelta(minutes=30 * i) for i in range(len(df2))]
# 4. CONFIGURACIÓN DE SERIE TEMPORAL
df2 = df2.set_index('fecha')
df2 = df2.sort_index()
df2 = df2.asfreq('30min')

# Convertir TSF y ABA% a float
df2['Real TSF']  = df2['Real TSF'].str.replace('%','').astype(float) / 100
df2['Real ABA%'] = df2['Real ABA%'].str.replace('%','').astype(float) / 100


# Asegurar el resto de columnas están numéricas inicialmente
df2['OFF Calls']     = df2['OFF Calls'].astype(int)
df2['Handled Calls'] = df2['Handled Calls'].astype(int)
df2['Real AHT']      = df2['Real AHT'].astype(int)
df2['Real ASA']      = df2['Real ASA'].astype(int)
df2['Real ABA']      = df2['Real ABA'].astype(int)

df2 = df2[cols]

# --- 3. Unificación ---
cutoff   = df2.index.max() + pd.Timedelta(minutes=30)
df1_tail = df1[df1.index >= cutoff]
df_final = pd.concat([df2, df1_tail])

# --- 4. Downcast de enteros para ahorrar espacio ---
int_cols = ['OFF Calls','Handled Calls','Real AHT','Real ASA','Real ABA']
df_final[int_cols] = df_final[int_cols].apply(pd.to_numeric, downcast='integer')
df_final[['Real TSF','Real ABA%']] = (df_final[['Real TSF','Real ABA%']] * 100).round(1)

# --- 5. Guardar y mostrar resultados ---
merged_path = 'merged_output.csv'
df_final.to_csv(merged_path)
print("Archivo unificado guardado como:", merged_path)


In [ ]:
df = pd.read_csv('merged_output.csv' , parse_dates=['fecha'], index_col='fecha')
df.index = pd.to_datetime(df.index)
df = df.asfreq('30min')


In [ ]:
df.head(80000)

In [ ]:
df.index

In [ ]:
df.dtypes

In [ ]:
# 2) Conteo de nulos por columna
null_counts = df.isna().sum().reset_index()
null_counts.columns = ['Column', 'Null Count']
print("Null counts per column:")
print(null_counts.to_string(index=False))

# 3) Filas con al menos un nulo
nan_rows = df[df.isna().any(axis=1)]
print("\nRows containing any nulls:")
print(nan_rows)


In [ ]:
for col in ['TSF','ABA %','AHT']:
    df_clean[col] = df_clean[col].interpolate(
        method='time',       # usa la distancia en el índice datetime
        limit_direction='both'
    )


In [ ]:
# 2. Filtro a partir del 1-Oct-2024
df_filt = df[df.index >= '2024-10-01']

# 3. Redondeo TSF y ABA% a 1 decimal
df_filt['Real TSF']  = df_filt['Real TSF'].round(1)
df_filt['Real ABA%'] = df_filt['Real ABA%'].round(1)

# 4. Agrupo y cuento cada par
pair_counts = (
    df_filt
    .groupby(['Real TSF', 'Real ABA%'])
    .size()
    .reset_index(name='Count')
    # Ordeno por Real TSF ascendente (y, de paso, por Real ABA% ascendente)
    .sort_values(['Real TSF'])
    .reset_index(drop=True)
)

# 5. Muestro el resultado
print(pair_counts.to_string(index=False))

In [ ]:
# Verify that a temporary index is complete
# ==============================================================================
start_date = df.index.min()
end_date = df.index.max()
complete_date_range = pd.date_range(start=start_date, end=end_date, freq=df.index.freq)
is_index_complete = (df.index == complete_date_range).all()
print(f"Index complete: {is_index_complete}")
print(f"Number of rows with missing values: {df.isnull().any(axis=1).mean()}")

In [ ]:
import numpy as np

# matriz booleana de NaNs
mask = df.isna().values
rows, cols = np.where(mask)

for r, c in zip(rows, cols):
    fecha = df.index[r]
    columna = df.columns[c]
    print(f"NaN en fila índice={r} (fecha={fecha}), columna='{columna}'")


In [ ]:
df.head()

In [ ]:
print("Puntos faltantes:", df.isna().sum().sum())

In [ ]:
df.head(80000)

In [ ]:
df.dtypes

In [ ]:
df = df.loc['2023-01-01 00:00:00':'2025-06-30 23:59:00'].copy()

In [ ]:
df.index = pd.to_datetime(df.index)

In [ ]:
df.head(80000)

In [ ]:
# ==== División de serie temporal sin solapamientos ====
def dividir_serie_temporal(df, col, end_train, end_validation):
    """
    Divide una columna de un DataFrame indexado por fecha en tres subconjuntos:
    entrenamiento (hasta end_train), validación (después de end_train hasta end_validation),
    y prueba (después de end_validation), sin solapamientos.
    """
    serie = df[col]
    # Asegurar tipos datetime
    end_train = pd.to_datetime(end_train)
    end_validation = pd.to_datetime(end_validation)
    # determinar frecuencia
    freq = df.index.freq or pd.infer_freq(df.index)
    # calcular pasos siguientes
    next_train = end_train + freq
    next_val = end_validation + freq

    data_train = serie.loc[:end_train].copy()
    data_val = serie.loc[next_train:end_validation].copy()
    data_test = serie.loc[next_val:].copy()

    print(f"📊 {col} - Train      : {data_train.index.min()} — {data_train.index.max()}  (n={len(data_train)})")
    print(f"📊 {col} - Validation : {data_val.index.min()} — {data_val.index.max()}  (n={len(data_val)})")
    print(f"📊 {col} - Test       : {data_test.index.min()} — {data_test.index.max()}  (n={len(data_test)})")

    return data_train, data_val, data_test


# ==== Gráfica de particiones con líneas verticales y retorno de figura ====
def graficar_particiones_temporales(data_train, data_val, data_test,
                                    variable, end_train, end_validation):
    """
    Grafica las particiones train/validation/test con líneas que marcan las fronteras.
    Devuelve el objeto fig para guardado o modificación posterior.
    """
    set_dark_theme()
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data_train.index, y=data_train,
                             mode='lines', name='Train'))
    fig.add_trace(go.Scatter(x=data_val.index, y=data_val,
                             mode='lines', name='Validation'))
    fig.add_trace(go.Scatter(x=data_test.index, y=data_test,
                             mode='lines', name='Test'))
    # Líneas de partición
    fig.add_vline(x=pd.to_datetime(end_train), line=dict(color='gray', dash='dash'))
    fig.add_vline(x=pd.to_datetime(end_validation), line=dict(color='gray', dash='dash'))

    fig.update_layout(
        title=f'Intraday 30-minutes {variable}',
        xaxis_title="Time",
        yaxis_title=variable,
        legend_title="Partition:",
        width=800, height=400,
        margin=dict(l=20, r=20, t=35, b=20),
        legend=dict(orientation="h", yanchor="top", y=1, xanchor="left", x=0.001)
    )
    fig.show()
    return fig


# ==== Prueba de estacionariedad con interpretación de p-value ====
def probar_estacionariedad(df, columna, alpha=0.05):
    """
    Aplica pruebas ADF y KPSS a la serie y sus primeras dos diferencias,
    e interpreta si cada serie es estacionaria según alpha.
    """
    serie = df[columna].dropna()
    data_diff_1 = serie.diff().dropna()
    data_diff_2 = data_diff_1.diff().dropna()

    for data, orden in zip([serie, data_diff_1, data_diff_2], [0,1,2]):
        adf = adfuller(data)
        kpss_ = kpss(data, nlags="auto")
        if orden == 0:
            print(f"\n📈 Serie original:")
        else:
            print(f"\n🔁 Diferenciada orden {orden}:")
        print(f"ADF  → estadístico: {adf[0]:.4f}, p-valor: {adf[1]:.4f} "
              f"-> {'Estacionaria' if adf[1]<alpha else 'No estacionaria'}")
        print(f"KPSS → estadístico: {kpss_[0]:.4f}, p-valor: {kpss_[1]:.4f} "
              f"-> {'No estacionaria' if kpss_[1]<alpha else 'Estacionaria'}")

    # Gráficas
    fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(7,5), sharex=True)
    serie.plot(ax=axs[0], title='Serie original')
    data_diff_1.plot(ax=axs[1], title='Diferenciada orden 1')
    data_diff_2.plot(ax=axs[2], title='Diferenciada orden 2')
    plt.tight_layout()
    plt.show()



# ==== Gráfica completa y zoom robusto ====
def graficar_con_zoom(df, columna, zoom_range):
    """
    Grafica la serie completa y un zoom en zoom_range usando máscara para fill_between.
    """
    serie = df[columna]
    inicio, fin = pd.to_datetime(zoom_range[0]), pd.to_datetime(zoom_range[1])

    fig = plt.figure(figsize=(8,4))
    grid = GridSpec(nrows=8, ncols=1, hspace=0.6, wspace=0)

    main_ax = fig.add_subplot(grid[:3,:])
    serie.plot(ax=main_ax, linewidth=0.5, alpha=0.5)
    mask = (serie.index >= inicio) & (serie.index <= fin)
    main_ax.fill_between(serie.index[mask], serie.min(), serie.max(),
                         facecolor='blue', alpha=0.5, zorder=0)
    main_ax.set_title(
        f"{columna} - Serie completa: {serie.index.min().date()} a {serie.index.max().date()}")

    zoom_ax = fig.add_subplot(grid[5:,:])
    serie.loc[inicio:fin].plot(ax=zoom_ax, linewidth=1)
    zoom_ax.set_title(f"{columna} - Zoom: {inicio} a {fin}")

    plt.subplots_adjust(hspace=1)
    plt.show()




# ==== Boxplots estacionales con etiquetas legibles ====
def graficar_boxplots_estacionalidad(df, columna):
    """
    Boxplots por mes, día de la semana (con etiquetas) y hora, más promedio por día-hora.
    """
    df_aux = df[[columna]].dropna().copy()
    df_aux['month'] = df_aux.index.month
    df_aux['week_day'] = df_aux.index.dayofweek  # 0=Lun
    df_aux['hour_day'] = df_aux.index.hour

    # mensual
    fig, ax = plt.subplots(figsize=(7,3))
    df_aux.boxplot(column=columna, by='month', ax=ax,
                   flierprops={'markersize':3, 'alpha':0.3})
    df_aux.groupby('month')[columna].median().plot(style='o-', linewidth=0.8, ax=ax)
    ax.set_title(f'{columna} - Distribución mensual')
    ax.set_xlabel('Mes')
    plt.suptitle('')
    plt.tight_layout()
    plt.show()

    # semanal
    fig, ax = plt.subplots(figsize=(7,3))
    df_aux.boxplot(column=columna, by='week_day', ax=ax,
                   flierprops={'markersize':3, 'alpha':0.3})
    day_labels = ["Lun","Mar","Mié","Jue","Vie","Sáb","Dom"]
    ax.set_xticklabels(day_labels)
    ax.set_title(f'{columna} - Distribución semanal')
    ax.set_ylabel(columna)
    plt.suptitle('')
    plt.tight_layout()
    plt.show()

    # horaria
    fig, ax = plt.subplots(figsize=(7,3))
    df_aux.boxplot(column=columna, by='hour_day', ax=ax,
                   flierprops={'markersize':3, 'alpha':0.3})
    ax.set_title(f'{columna} - Distribución horaria')
    ax.set_xlabel('Hora del día')
    plt.suptitle('')
    plt.tight_layout()
    plt.show()

    # promedio día-hora
    fig, ax = plt.subplots(figsize=(7,3))
    mean_dh = df_aux.groupby(['week_day','hour_day'])[columna].mean()
    mean_dh.unstack(level=0).plot(ax=ax)
    ax.set_title(f"{columna} - Promedio por hora y día")
    ax.set_xlabel("Hora del día")
    ax.set_ylabel(columna)
    plt.tight_layout()
    plt.show()




# ==== Descomposición STL múltiple y retorno de componentes ====
def descomponer_stl(df, columna, periodos, rango=None):
    """
    Descompone la serie con STL usando uno o varios periodos.
    Retorna un dict de resultados con trend, seasonal y resid para cada periodo.
    """
    serie = df[columna].dropna()
    if rango:
        inicio, fin = pd.to_datetime(rango[0]), pd.to_datetime(rango[1])
        serie = serie.loc[inicio:fin]
    if not isinstance(periodos, (list,tuple)):
        periodos = [periodos]

    resultados = {}
    for periodo in periodos:
        stl = STL(serie, period=periodo)
        res = stl.fit()
        resultados[periodo] = res
        fig = res.plot()
        fig.suptitle(f"STL period={periodo}", fontsize=12)
        plt.tight_layout()
        plt.show()
    return resultados




# ==== Test general de estacionariedad ====
def test_estacionariedad(df, columna, rango=None, alpha=0.05):
    """
    ADF y KPSS para toda la serie o un rango específico, con interpretación.
    """
    if rango:
        inicio, fin = pd.to_datetime(rango[0]), pd.to_datetime(rango[1])
        serie = df[columna].loc[inicio:fin].dropna()
        print(f"\n📈 Estacionariedad de '{columna}' entre {inicio} y {fin}")
    else:
        serie = df[columna].dropna()
        print(f"\n📈 Estacionariedad de toda la serie '{columna}'")

    adf = adfuller(serie)
    kpss_ = kpss(serie, nlags="auto")
    print(f"ADF   → Estadístico: {adf[0]:.4f}, p-value: {adf[1]:.4f} "
          f"-> {'Estacionaria' if adf[1]<alpha else 'No estacionaria'}")
    print(f"KPSS  → Estadístico: {kpss_[0]:.4f}, p-value: {kpss_[1]:.4f} "
          f"-> {'No estacionaria' if kpss_[1]<alpha else 'Estacionaria'}")

In [ ]:
# Ejemplo de llamada:
train, val, test = dividir_serie_temporal(df, 'OFF Calls', '2024-10-31 23:59:00', '2025-05-30 23:59:00')
fig = graficar_particiones_temporales(train, val, test, 'OFF Calls', '2024-10-31 23:59:00', '2025-05-30 23:59:00')
probar_estacionariedad(df, 'OFF Calls', alpha=0.05)
graficar_con_zoom(df, 'OFF Calls',('2024-05-01 14:00:00', '2024-06-01 14:00:00'))
graficar_boxplots_estacionalidad(df, 'OFF Calls')
res_dict = descomponer_stl(df, 'OFF Calls', [48, 48*7], rango=('2024-05-01 14:00:00', '2024-06-01 14:00:00'))
test_estacionariedad(df, 'OFF Calls', rango=('2024-05-01 14:00:00','2024-06-01 14:00:00'), alpha=0.05)

In [ ]:
# Handled Calls
train, val, test = dividir_serie_temporal(df, 'Handled Calls', '2024-10-31 23:59:00', '2025-05-31 23:59:00')
fig = graficar_particiones_temporales(train, val, test, 'Handled Calls', '2024-10-31 23:59:00', '2025-05-31 23:59:00')
probar_estacionariedad(df, 'Handled Calls', alpha=0.05)
graficar_con_zoom(df, 'Handled Calls', ('2024-05-01 14:00:00', '2024-06-01 14:00:00'))
graficar_boxplots_estacionalidad(df, 'Handled Calls')
res_dict = descomponer_stl(df, 'Handled Calls', [48, 48*7], rango=('2024-05-01 14:00:00', '2024-06-01 14:00:00'))
test_estacionariedad(df, 'Handled Calls', rango=('2024-05-01 14:00:00', '2024-06-01 14:00:00'), alpha=0.05)

In [ ]:
# Real AHT
train, val, test = dividir_serie_temporal(df, 'Real AHT', '2024-10-31 23:59:00', '2025-05-31 23:59:00')
fig = graficar_particiones_temporales(train, val, test, 'Real AHT', '2024-10-31 23:59:00', '2025-05-31 23:59:00')
probar_estacionariedad(df, 'Real AHT', alpha=0.05)
graficar_con_zoom(df, 'Real AHT', ('2024-05-01 14:00:00', '2024-06-01 14:00:00'))
graficar_boxplots_estacionalidad(df, 'Real AHT')
res_dict = descomponer_stl(df, 'Real AHT', [48, 48*7], rango=('2024-05-01 14:00:00', '2024-06-01 14:00:00'))
test_estacionariedad(df, 'Real AHT', rango=('2024-05-01 14:00:00', '2024-06-01 14:00:00'), alpha=0.05)

In [ ]:
# Real TSF
train, val, test = dividir_serie_temporal(df, 'Real TSF', '2024-10-31 23:59:00', '2025-05-31 23:59:00')
fig = graficar_particiones_temporales(train, val, test, 'Real TSF', '2024-10-31 23:59:00', '2025-05-31 23:59:00')
probar_estacionariedad(df, 'Real TSF', alpha=0.05)
graficar_con_zoom(df, 'Real TSF', ('2024-05-01 14:00:00', '2024-06-01 14:00:00'))
graficar_boxplots_estacionalidad(df, 'Real TSF')
res_dict = descomponer_stl(df, 'Real TSF', [48, 48*7], rango=('2024-05-01 14:00:00', '2024-06-01 14:00:00'))
test_estacionariedad(df, 'Real TSF', rango=('2024-05-01 14:00:00', '2024-06-01 14:00:00'), alpha=0.05)

In [ ]:
# Real ABA
train, val, test = dividir_serie_temporal(df, 'Real ABA', '2024-10-31 23:59:00', '2025-05-31 23:59:00')
fig = graficar_particiones_temporales(train, val, test, 'Real ABA', '2024-10-31 23:59:00', '2025-05-31 23:59:00')
probar_estacionariedad(df, 'Real ABA', alpha=0.05)
graficar_con_zoom(df, 'Real ABA', ('2024-05-01 14:00:00', '2024-06-01 14:00:00'))
graficar_boxplots_estacionalidad(df, 'Real ABA')
res_dict = descomponer_stl(df, 'Real ABA', [48, 48*7], rango=('2024-05-01 14:00:00', '2024-06-01 14:00:00'))
test_estacionariedad(df, 'Real ABA', rango=('2024-05-01 14:00:00', '2024-06-01 14:00:00'), alpha=0.05)

In [ ]:
# Real ASA
train, val, test = dividir_serie_temporal(df, 'Real ASA', '2024-10-31 23:59:00', '2025-05-31 23:59:00')
fig = graficar_particiones_temporales(train, val, test, 'Real ASA', '2024-10-31 23:59:00', '2025-05-31 23:59:00')
probar_estacionariedad(df, 'Real ASA', alpha=0.05)
graficar_con_zoom(df, 'Real ASA', ('2024-05-01 14:00:00', '2024-06-01 14:00:00'))
graficar_boxplots_estacionalidad(df, 'Real ASA')
res_dict = descomponer_stl(df, 'Real ASA', [48, 48*7], rango=('2024-05-01 14:00:00', '2024-06-01 14:00:00'))
test_estacionariedad(df, 'Real ASA', rango=('2024-05-01 14:00:00', '2024-06-01 14:00:00'), alpha=0.05)

In [ ]:
df.head()

#### 2. Autocorrelacion y autocorrealcion parcial de targets

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from statsmodels.tsa.stattools import adfuller, kpss, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import STL
import plotly.graph_objects as go
from skforecast.plot import set_dark_theme
from matplotlib.gridspec import GridSpec

# ==== División de serie temporal sin solapamientos ====
# ... (funciones previas mantendrían su lugar)


# ==== Autocorrelación y Autocorrelación Parcial ====
def plot_autocorrelation(data, columna_objetivo, orden_diff=0, lags=120, figsize=(5, 2)):
    """
    Grafica la función de autocorrelación (ACF) para `columna_objetivo`,
    opcionalmente diferenciada `orden_diff` veces.
    """
    serie = data[columna_objetivo].dropna()
    if orden_diff > 0:
        serie = serie.diff(periods=orden_diff).dropna()

    fig, ax = plt.subplots(figsize=figsize)
    plot_acf(serie, ax=ax, lags=lags)
    ax.set_title(f"ACF - {columna_objetivo} (diff={orden_diff})")
    plt.tight_layout()
    plt.show()
    return fig


def plot_partial_autocorrelation(data, columna_objetivo, orden_diff=0, lags=120, figsize=(5, 2)):
    """
    Grafica la función de autocorrelación parcial (PACF) para `columna_objetivo`,
    opcionalmente diferenciada `orden_diff` veces.
    """
    serie = data[columna_objetivo].dropna()
    if orden_diff > 0:
        serie = serie.diff(periods=orden_diff).dropna()

    fig, ax = plt.subplots(figsize=figsize)
    plot_pacf(serie, ax=ax, lags=lags, method='ywm')
    ax.set_title(f"PACF - {columna_objetivo} (diff={orden_diff})")
    plt.tight_layout()
    plt.show()
    return fig


def calcular_lag_autocorrelaciones(
    serie: pd.Series,
    n_lags: int = 120,
    orden_diff: int = 0,
    sort_by: str = "partial_autocorrelation_abs"
) -> pd.DataFrame:
    """
    Calcula ACF y PACF de `serie` con `n_lags`, tras diferenciar `orden_diff` veces.

    Retorna DataFrame ordenado según `sort_by`.
    """
    if not isinstance(serie, pd.Series):
        raise ValueError("La entrada 'serie' debe ser una pd.Series")

    s = serie.dropna()
    if orden_diff > 0:
        s = s.diff(periods=orden_diff).dropna()

    acf_vals = acf(s, nlags=n_lags)
    pacf_vals = pacf(s, nlags=n_lags, method='ywm')

    df_lags = pd.DataFrame({
        "lag": range(len(acf_vals)),
        "autocorrelation": acf_vals,
        "partial_autocorrelation": pacf_vals
    })
    df_lags["autocorrelation_abs"] = df_lags["autocorrelation"].abs()
    df_lags["partial_autocorrelation_abs"] = df_lags["partial_autocorrelation"].abs()
    df_lags = df_lags[df_lags["lag"] > 0]

    if sort_by in df_lags.columns:
        df_lags = df_lags.sort_values(by=sort_by, ascending=False)

    return df_lags.reset_index(drop=True)

In [ ]:
# ==== Ejemplos de uso ====
fig_acf = plot_autocorrelation(df, 'OFF Calls', orden_diff=0, lags=120)
fig_pacf = plot_partial_autocorrelation(df, 'OFF Calls', orden_diff=0, lags=120)
top_lags = calcular_lag_autocorrelaciones(df['OFF Calls'], n_lags=120, orden_diff=0, sort_by="partial_autocorrelation_abs")
print(top_lags.head(10))

In [ ]:
# ==== Ejemplos de uso ====
fig_acf = plot_autocorrelation(df, 'OFF Calls', orden_diff=1, lags=120)
fig_pacf = plot_partial_autocorrelation(df, 'OFF Calls', orden_diff=1, lags=120)
top_lags = calcular_lag_autocorrelaciones(df['OFF Calls'], n_lags=120, orden_diff=1, sort_by="partial_autocorrelation_abs")
print(top_lags.head(10))

In [ ]:
# ==== Ejemplos de uso ====
fig_acf = plot_autocorrelation(df, 'OFF Calls', orden_diff=2, lags=120)
fig_pacf = plot_partial_autocorrelation(df, 'OFF Calls', orden_diff=2, lags=120)
top_lags = calcular_lag_autocorrelaciones(df['OFF Calls'], n_lags=120, orden_diff=2, sort_by="partial_autocorrelation_abs")
print(top_lags.head(10))

In [ ]:
# ==== Ejemplos de uso ====
fig_acf = plot_autocorrelation(df, 'Handled Calls', orden_diff=0, lags=120)
fig_pacf = plot_partial_autocorrelation(df, 'Handled Calls', orden_diff=0, lags=120)
top_lags = calcular_lag_autocorrelaciones(df['Handled Calls'], n_lags=120, orden_diff=0, sort_by="partial_autocorrelation_abs")
print(top_lags.head(10))

In [ ]:
# ==== Ejemplos de uso ====
fig_acf = plot_autocorrelation(df, 'Handled Calls', orden_diff=1, lags=120)
fig_pacf = plot_partial_autocorrelation(df, 'Handled Calls', orden_diff=1, lags=120)
top_lags = calcular_lag_autocorrelaciones(df['Handled Calls'], n_lags=120, orden_diff=1, sort_by="partial_autocorrelation_abs")
print(top_lags.head(10))

In [ ]:
# ==== Ejemplos de uso ====
fig_acf = plot_autocorrelation(df, 'Handled Calls', orden_diff=2, lags=120)
fig_pacf = plot_partial_autocorrelation(df, 'Handled Calls', orden_diff=2, lags=120)
top_lags = calcular_lag_autocorrelaciones(df['Handled Calls'], n_lags=120, orden_diff=2, sort_by="partial_autocorrelation_abs")
print(top_lags.head(10))

In [ ]:
# ==== Ejemplos de uso ====
fig_acf = plot_autocorrelation(df, 'Real AHT', orden_diff=0, lags=120)
fig_pacf = plot_partial_autocorrelation(df, 'Real AHT', orden_diff=0, lags=120)
top_lags = calcular_lag_autocorrelaciones(df['Real AHT'], n_lags=120, orden_diff=0, sort_by="partial_autocorrelation_abs")
print(top_lags.head(10))

In [ ]:
# ==== Ejemplos de uso ====
fig_acf = plot_autocorrelation(df, 'Real AHT', orden_diff=1, lags=120)
fig_pacf = plot_partial_autocorrelation(df, 'Real AHT', orden_diff=1, lags=120)
top_lags = calcular_lag_autocorrelaciones(df['Real AHT'], n_lags=120, orden_diff=1, sort_by="partial_autocorrelation_abs")
print(top_lags.head(10))

In [ ]:
# ==== Ejemplos de uso ====
fig_acf = plot_autocorrelation(df, 'Real AHT', orden_diff=2, lags=120)
fig_pacf = plot_partial_autocorrelation(df, 'Real AHT', orden_diff=2, lags=120)
top_lags = calcular_lag_autocorrelaciones(df['Real AHT'], n_lags=120, orden_diff=2, sort_by="partial_autocorrelation_abs")
print(top_lags.head(10))

In [ ]:
# ==== Ejemplos de uso ====
fig_acf = plot_autocorrelation(df, 'Real TSF', orden_diff=0, lags=120)
fig_pacf = plot_partial_autocorrelation(df, 'Real TSF', orden_diff=0, lags=120)
top_lags = calcular_lag_autocorrelaciones(df['Real TSF'], n_lags=120, orden_diff=0, sort_by="partial_autocorrelation_abs")
print(top_lags.head(10))

In [ ]:
# ==== Ejemplos de uso ====
fig_acf = plot_autocorrelation(df, 'Real TSF', orden_diff=1, lags=120)
fig_pacf = plot_partial_autocorrelation(df, 'Real TSF', orden_diff=1, lags=120)
top_lags = calcular_lag_autocorrelaciones(df['Real TSF'], n_lags=120, orden_diff=1, sort_by="partial_autocorrelation_abs")
print(top_lags.head(10))

In [ ]:
# ==== Ejemplos de uso ====
fig_acf = plot_autocorrelation(df, 'Real TSF', orden_diff=2, lags=120)
fig_pacf = plot_partial_autocorrelation(df, 'Real TSF', orden_diff=2, lags=120)
top_lags = calcular_lag_autocorrelaciones(df['Real TSF'], n_lags=120, orden_diff=2, sort_by="partial_autocorrelation_abs")
print(top_lags.head(10))

In [ ]:
# ==== Ejemplos de uso ====
fig_acf = plot_autocorrelation(df, 'Real ABA', orden_diff=0, lags=120)
fig_pacf = plot_partial_autocorrelation(df, 'Real ABA', orden_diff=0, lags=120)
top_lags = calcular_lag_autocorrelaciones(df['Real ABA'], n_lags=120, orden_diff=0, sort_by="partial_autocorrelation_abs")
print(top_lags.head(10))

In [ ]:
# ==== Ejemplos de uso ====
fig_acf = plot_autocorrelation(df, 'Real ABA', orden_diff=1, lags=120)
fig_pacf = plot_partial_autocorrelation(df, 'Real ABA', orden_diff=1, lags=120)
top_lags = calcular_lag_autocorrelaciones(df['Real ABA'], n_lags=120, orden_diff=1, sort_by="partial_autocorrelation_abs")
print(top_lags.head(10))

In [ ]:
# ==== Ejemplos de uso ====
fig_acf = plot_autocorrelation(df, 'Real ABA', orden_diff=2, lags=120)
fig_pacf = plot_partial_autocorrelation(df, 'Real ABA', orden_diff=2, lags=120)
top_lags = calcular_lag_autocorrelaciones(df['Real ABA'], n_lags=120, orden_diff=2, sort_by="partial_autocorrelation_abs")
print(top_lags.head(10))

In [ ]:
# ==== Ejemplos de uso ====
fig_acf = plot_autocorrelation(df, 'Real ASA', orden_diff=0, lags=120)
fig_pacf = plot_partial_autocorrelation(df, 'Real ASA', orden_diff=0, lags=120)
top_lags = calcular_lag_autocorrelaciones(df['Real ASA'], n_lags=120, orden_diff=0, sort_by="partial_autocorrelation_abs")
print(top_lags.head(10))

In [ ]:
# ==== Ejemplos de uso ====
fig_acf = plot_autocorrelation(df, 'Real ASA', orden_diff=1, lags=120)
fig_pacf = plot_partial_autocorrelation(df, 'Real ASA', orden_diff=1, lags=120)
top_lags = calcular_lag_autocorrelaciones(df['Real ASA'], n_lags=120, orden_diff=1, sort_by="partial_autocorrelation_abs")
print(top_lags.head(10))

In [ ]:
# ==== Ejemplos de uso ====
fig_acf = plot_autocorrelation(df, 'Real ASA', orden_diff=2, lags=120)
fig_pacf = plot_partial_autocorrelation(df, 'Real ASA', orden_diff=2, lags=120)
top_lags = calcular_lag_autocorrelaciones(df['Real ASA'], n_lags=120, orden_diff=2, sort_by="partial_autocorrelation_abs")
print(top_lags.head(10))

### 3. Baseline antes de usar modelos complejos

#### Seasonal Naive Forecasting and backtesting

In [ ]:
df.index = pd.to_datetime(df.index)

In [ ]:
df.index

In [ ]:
import pandas as pd
from skforecast.recursive import ForecasterEquivalentDate
from skforecast.model_selection import TimeSeriesFold, backtesting_forecaster

def baseline_naive_forecast(
    df: pd.DataFrame,
    column: str,
    end_validation: str,
    offset: pd.DateOffset = pd.DateOffset(days=1),
    steps: int = 48,
    metric: str = 'mean_absolute_error'
):
    """
    Construye un baseline naive (valor del mismo instante del día anterior),
    evalúa su MAE con backtesting y devuelve la métrica y las predicciones.
    """
    # 1. Preparar y ajustar el forecaster
    y = df[column].dropna()
    y_train = y.loc[: end_validation]
    forecaster = ForecasterEquivalentDate(
        offset    = offset,
        n_offsets = 1
    )
    forecaster.fit(y = y_train)

    # 2. Configurar el CV
    cv = TimeSeriesFold(
        initial_train_size = len(y_train),
        steps              = steps,
        refit              = False
    )

    # 3. Ejecutar backtesting
    metric_value, predictions = backtesting_forecaster(
        forecaster = forecaster,
        y          = y,
        cv         = cv,
        metric     = metric
    )

    # 4. Asegurar un valor numérico escalar de la métrica
    try:
        metric_float = float(metric_value)
    except Exception:
        # Si es Series o DataFrame, tomamos la media de sus valores
        if hasattr(metric_value, 'mean'):
            metric_float = metric_value.mean() if not isinstance(metric_value, pd.DataFrame) \
                            else metric_value.values.mean()
        else:
            # fallback genérico
            metric_float = sum(metric_value) / len(metric_value)

    print(f"✅ Baseline naive [{column}] → {metric.upper()} = {metric_float:.2f}")
    return metric_float, predictions

In [ ]:
targets = ['OFF Calls', 'Handled Calls', 'Real AHT', 'Real ABA', 'Real TSF', 'Real ASA']
end_val = '2025-05-31 23:59:00'
results = {}

for col in targets:
    mae, preds = baseline_naive_forecast(
        df,
        column         = col,
        end_validation = end_val,
        offset         = pd.DateOffset(days=1),
        steps          = 48,
        metric         = 'mean_absolute_error'
    )
    results[col] = mae

print("\nMAE summary:")
for col, mae in results.items():
    print(f" - {col:12}: {mae:.2f}")

## 5. Empezando modelo para OFF Calls

#### Multi\-Step Recursive Forecasting with backtesting



In [ ]:
data_train, data_val, data_test = dividir_serie_temporal(df, 'OFF Calls', '2024-10-31 23:59:00', '2025-05-31 23:59:00')

In [ ]:
end_train = '2024-10-31 23:59:00'
end_validation = '2025-05-31 23:59:00'

In [ ]:
# Create forecaster
# ==============================================================================
window_features = RollingFeatures(stats=["mean"], window_sizes=48 * 3)
forecaster = ForecasterRecursive(
                 regressor       = LGBMRegressor(random_state=15926, verbose=-1),
                 lags            = 48,
                 window_features = window_features
             )

# Train forecaster
# ==============================================================================
forecaster.fit(y=df.loc[:end_validation, 'OFF Calls'])
forecaster

In [ ]:

cv = TimeSeriesFold(
        steps              = 48,
        initial_train_size = len(df.loc[:end_validation]),
        refit              = False
)
# Backtesting
# ==============================================================================
metric, predictions = backtesting_forecaster(
                          forecaster    = forecaster,
                          y             = df['OFF Calls'],
                          cv            = cv,
                          metric        = 'mean_absolute_error',
                          verbose       = True, # Set to False to avoid printing
                      )

In [ ]:
metric

In [ ]:
df.head()

In [ ]:
import pandas as pd
import random

def obtener_muestras_semanales_por_anos(
    df: pd.DataFrame,
    columna_objetivo: str = 'OFF Calls',
    anos: list = None,
    semanas: int = 4,
    seed: int = 42
) -> list:
    """
    Extrae muestras de longitud `semanas` semanas para la misma fecha inicio
    (mes, día, hora, minuto), pero en diferentes años especificados en `anos`,
    ajustándose al rango disponible en el DataFrame.

    Parámetros:
    - df: DataFrame indexado en datetime.
    - columna_objetivo: nombre de la columna a muestrear.
    - anos: lista de años (int) para generar cada muestra.
    - semanas: duración de cada muestra en semanas.
    - seed: semilla para reproducibilidad.

    Retorna:
    - Listado de dicts con 'inicio', 'fin' y 'valores'.
    """
    # Copiar y ordenar
    df2 = df.copy()
    df2.index = pd.to_datetime(df2.index)
    df2 = df2.sort_index()

    # Filtrar años disponibles
    años_disponibles = sorted(df2.index.year.unique())
    if anos is None:
        anos = años_disponibles
    else:
        anos = [y for y in anos if y in años_disponibles]
    if not anos:
        raise ValueError("No hay años disponibles en el DataFrame para las muestras solicitadas.")

    # Inferir frecuencia y duraciones
    freq = df2.index.freq or pd.infer_freq(df2.index)
    freq_delta = pd.to_timedelta(freq)
    delta = pd.Timedelta(weeks=semanas)

    # Índice del primer año y rango válido de inicio
    primer_ano = anos[0]
    idx_first = df2.loc[str(primer_ano)].index
    start_bound = idx_first.min()
    end_bound = idx_first.max() - delta + freq_delta
    valid_starts = idx_first[(idx_first >= start_bound) & (idx_first <= end_bound)]
    if valid_starts.empty:
        raise ValueError("No hay rangos de inicio válidos para el primer año.")

    # Seleccionar un inicio que exista en todos los años
    random.seed(seed)
    candidate = None
    for _ in range(1000):
        ts = random.choice(valid_starts)
        fin = ts + delta - freq_delta
        if all(
            (ts.replace(year=y) >= df2.loc[str(y)].index.min()) and
            (fin.replace(year=y) <= df2.loc[str(y)].index.max())
            for y in anos
        ):
            candidate = ts
            break
    if candidate is None:
        raise ValueError("No se encontró inicio válido tras múltiples intentos.")

    # Generar muestras para cada año
    muestras = []
    for y in anos:
        inicio = candidate.replace(year=y)
        fin = inicio + delta - freq_delta
        valores = df2.loc[inicio:fin, columna_objetivo].tolist()
        muestras.append({'inicio': inicio, 'fin': fin, 'valores': valores})

    return muestras


def mostrar_muestras(muestras: list) -> None:
    """
    Imprime cada muestra con detalles de fechas y valores.
    """
    for i, m in enumerate(muestras, 1):
        print(f"📦 Muestra {i}")
        print(f"   ⏱️ Desde: {m['inicio']}  →  Hasta: {m['fin']}")
        print(f"   🔢 Valores (primeros 10): {m['valores'][:10]}")
        print(f"   🔢 Valores (todos): {m['valores']}\n")

In [ ]:
# Ejemplo:
muestras = obtener_muestras_semanales_por_anos(
    df,
    columna_objetivo='OFF Calls',
    anos=[2023,2024,2025],
    semanas=4,
    seed=42
)
mostrar_muestras(muestras)

In [ ]:
import pandas as pd
import numpy as np
from feature_engine.datetime import DatetimeFeatures
from feature_engine.creation import CyclicalFeatures
# (Optional) define holiday list as in original

def crear_variables_exogenas_callcenter(df: pd.DataFrame, columna_objetivo: str) -> pd.DataFrame:
    """
    Crea variables exógenas para predicción de OFF Calls en un call center (frecuencia 30-min).
    Incluye:
    - Calendario y codificación cíclica
    - Feriados en Jamaica (2021–2025)
    - Variables semánticas (fin de semana, hora laboral, pico de horas)
    - Estadísticas rodantes sobre el objetivo (media, max, std, CV)
    - Indicadores de picos y zonas de trabajo (clusters)

    """
    df = df.copy()
    df.index = pd.to_datetime(df.index)
    y = df[columna_objetivo]

    # 1. Calendario
    calendar = DatetimeFeatures(
        variables='index',
        features_to_extract=['month', 'week', 'day_of_week', 'hour'],
        drop_original=True
    ).fit_transform(df)

    # 2. Cíclico
    cyc = CyclicalFeatures(
        variables=['month','week','day_of_week','hour'],
        max_values={'month':12,'week':52,'day_of_week':6,'hour':24},
        drop_original=False
    ).fit_transform(calendar)

    # 3. Feriados (simplificado al 2021-2025)
    feriados = pd.to_datetime([
        '2021-01-01',  # New Year's Day
        '2021-02-17',  # Ash Wednesday
        '2021-04-02',  # Good Friday
        '2021-04-04',  # Easter Day
        '2021-04-05',  # Easter Monday
        '2021-05-24',  # Labour Day (in lieu)
        '2021-08-01',  # Emancipation Day
        '2021-08-02',  # Emancipation Day (in lieu)
        '2021-08-06',  # Independence Day
        '2021-10-18',  # National Heroes' Day
        '2021-12-25',  # Christmas Day
        '2021-12-26',  # Boxing Day
        '2021-12-27',  # Christmas Day (in lieu)
        '2022-01-01',  # New Year's Day
        '2022-03-02',  # Ash Wednesday
        '2022-04-15',  # Good Friday
        '2022-04-17',  # Easter Day
        '2022-04-18',  # Easter Monday
        '2022-05-23',  # Labour Day
        '2022-08-01',  # Emancipation Day
        '2022-08-06',  # Independence Day
        '2022-10-17',  # National Heroes' Day
        '2022-12-25',  # Christmas Day
        '2022-12-26',  # Christmas Day (in lieu)
        '2022-12-27',  # Boxing Day (in lieu)
        '2023-01-01',  # New Year's Day
        '2023-02-22',  # Ash Wednesday
        '2023-04-07',  # Good Friday
        '2023-04-09',  # Easter Day
        '2023-04-10',  # Easter Monday
        '2023-05-23',  # Labour Day
        '2023-08-01',  # Emancipation Day
        '2023-08-06',  # Independence Day
        '2023-08-07',  # Independence Day (in lieu)
        '2023-10-16',  # National Heroes' Day
        '2023-12-25',  # Christmas Day
        '2023-12-26',  # Boxing Day
        '2024-01-01',  # New Year's Day
        '2024-02-14',  # Ash Wednesday
        '2024-03-29',  # Good Friday
        '2024-04-01',  # Easter Monday
        '2024-05-23',  # Labour Day
        '2024-08-01',  # Emancipation Day
        '2024-08-06',  # Independence Day
        '2024-10-21',  # National Heroes' Day
        '2024-12-25',  # Christmas Day
        '2024-12-26',  # Boxing Day
        '2025-01-01',  # New Year's Day
        '2025-03-05',  # Ash Wednesday
        '2025-04-18',  # Good Friday
        '2025-04-21',  # Easter Monday
        '2025-05-23',  # Labour Day
        '2025-08-01',  # Emancipation Day
        '2025-08-06',  # Independence Day
        '2025-10-21',  # National Heroes' Day
        '2025-12-25',  # Christmas Day
        '2025-12-26',  # Boxing Day
        '2026-01-01',  # New Year's Day
        '2026-02-18',  # Ash Wednesday
        '2026-04-03',  # Good Friday
        '2026-04-06',  # Easter Monday
        '2026-05-25',  # Labour Day (observed)
        '2026-08-01',  # Emancipation Day
        '2026-08-06',  # Independence Day
        '2026-10-19',  # National Heroes' Day
        '2026-12-25',  # Christmas Day
        '2026-12-26',  # Boxing Day
        '2027-02-10',  # Ash Wednesday
        '2027-03-26',  # Good Friday
        '2027-03-29',  # Easter Monday
        '2027-05-24',  # Labour Day (observed)
        '2027-08-02',  # Emancipation Day (observed)
        '2027-08-06',  # Independence Day
        '2027-10-18',  # National Heroes' Day
        '2027-12-25',  # Christmas Day
        '2027-12-26',  # Boxing Day
        '2027-12-27',  # Boxing Day (substitute)
        '2028-01-01',  # New Year's Day
        '2028-03-01',  # Ash Wednesday
        '2028-04-14',  # Good Friday
        '2028-04-17',  # Easter Monday
        '2028-05-23',  # Labour Day
        '2028-08-01',  # Emancipation Day
        '2028-08-06',  # Independence Day
        '2028-10-16',  # National Heroes' Day
        '2028-12-25',  # Christmas Day
        '2028-12-26',  # Boxing Day

    ])
    holiday = df.index.normalize().isin(feriados).astype(int)
    hol = pd.DataFrame({'holiday':holiday}, index=df.index)
    hol['prev_holiday'] = hol['holiday'].shift(48).fillna(0).astype(int)
    hol['next_holiday'] = hol['holiday'].shift(-48).fillna(0).astype(int)

    # 4. Estadísticas rodantes (rolling) SIN generar NaN en la primera fila
    windows = {'1D':48, '3D':48*3, '7D':48*7}
    roll = pd.DataFrame(index=df.index)
    for label, size in windows.items():
        roll[f'roll_mean_{label}'] = y.rolling(window=size, min_periods=1).mean()
        roll[f'roll_max_{label}']  = y.rolling(window=size, min_periods=1).max()
        # ddof=0 para que std con un solo punto sea 0, no NaN
        roll[f'roll_std_{label}']  = (
            y.rolling(window=size, min_periods=1)
             .std(ddof=0)
        )
        # coeficiente de variación, sin infinitos ni NaN
        roll[f'roll_cv_{label}']   = (
            roll[f'roll_std_{label}']
            .div( roll[f'roll_mean_{label}'].replace(0, np.nan) )
            .fillna(0)
        )

    # 5. Indicadores de picos y zonas
    q33, q66, q95 = np.percentile(y.dropna(), [33, 66, 95])
    zone = pd.cut(y, bins=[-np.inf, q33, q66, np.inf], labels=[0,1,2]).astype(int)
    peak = (y >= q95).astype(int)
    peaks = pd.DataFrame({'zone_cluster':zone, 'is_peak_event':peak}, index=df.index)

    # 6. Variables semánticas
    logic = pd.DataFrame(index=df.index)
    logic['is_weekend']      = df.index.dayofweek.isin([5,6]).astype(int)
    logic['is_working_hour'] = ((df.index.hour>=7)&(df.index.hour<18)).astype(int)
    logic['is_peak_hour']    = ((df.index.hour>=9)&(df.index.hour<16)).astype(int)

    # 7. Concatenar
    exog = pd.concat([cyc, hol, roll, peaks, logic], axis=1)

    # 8. Eliminar todas las columnas que vinieran de df (incluyendo otros targets)
    original_cols = df.columns.tolist()
    exog = exog.drop(columns=original_cols, errors='ignore')

    # Si quieres asegurarte de no generar NaNs imprevistos:
    exog = exog.fillna(0)
    return exog


In [ ]:
# 1. Importa (si está en otro fichero):
# from sampling_utils import crear_variables_exogenas_callcenter

# 2. Llama a la función pasándole tu DataFrame y la columna objetivo
exog = crear_variables_exogenas_callcenter(
    df=df,
    columna_objetivo='OFF Calls'
)

# 3. Comprueba el resultado
print(f"Shape de exógenas: {exog.shape}")
display(exog.head(80000))


In [ ]:
df.head(80000)
print(f"Shape de data: {df.shape}")

In [ ]:
df.head(80000)

In [ ]:
print(exog.columns)

In [ ]:
print('OFF Calls' in exog.columns)  # → False esperado

Estas sí las conoces por adelantado. Por ejemplo:
- Volumen planificado: turnos por agente
- Carga anticipada: reservas, sesiones agendadas, campañas
- Métricas históricas programadas: si sabes la cantidad de contactos estimados, o el canal más usado
- Condiciones repetitivas: si cada lunes a las 9am hay saturación, puedes modelarlo con calendario + lag.
⚙️ O usar lags autoregresiv


In [ ]:
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures

def crear_interacciones_exogenas(
    exog: pd.DataFrame,
    columnas_interaccion: list[str],
    degree: int = 2
) -> pd.DataFrame:
    """
    Añade interacciones de segundo orden (solo términos cruzados)
    a un DataFrame de variables exógenas.

    Parámetros
    ----------
    exog : pd.DataFrame
        DataFrame con variables exógenas indexado en datetime.
    columnas_interaccion : list[str]
        Lista de columnas de `exog` sobre las que generar interacciones.
    degree : int, opcional
        Grado del polinomio; por defecto 2 (interacciones de segundo orden).

    Retorna
    -------
    pd.DataFrame
        DataFrame original concatenado con las nuevas características de interacción.
    """
    # Configurar el transformador para que devuelva un DataFrame de pandas
    transformer = PolynomialFeatures(
        degree=degree,
        interaction_only=True,
        include_bias=False
    )
    transformer.set_output(transform="pandas")

    # Ajustar y transformar solo las columnas seleccionadas
    df_poly = transformer.fit_transform(exog[columnas_interaccion])

    # Eliminar las columnas originales si no las queremos duplicadas
    df_poly = df_poly.drop(columns=columnas_interaccion, errors='ignore')

    # Renombrar las columnas de interacción para identificarlas
    df_poly.columns = [
        f"poly__{col.replace(' ', '__')}" for col in df_poly.columns
    ]
    df_poly.index = exog.index

    # Concatenar con el DataFrame original
    exog_expanded = pd.concat([exog, df_poly], axis=1)
    return exog_expanded


In [ ]:
# 1. Define las columnas sobre las que quieres interacciones
poly_cols = [
    'month_sin','month_cos',
    'week_sin','week_cos',
    'day_of_week_sin','day_of_week_cos',
    'hour_sin','hour_cos',
    'holiday','prev_holiday','next_holiday',
    'roll_mean_1D','roll_max_1D','roll_std_1D','roll_cv_1D',
    'roll_mean_3D','roll_max_3D','roll_std_3D','roll_cv_3D',
    'roll_mean_7D','roll_max_7D','roll_std_7D','roll_cv_7D',
    'zone_cluster','is_peak_event',
    'is_weekend','is_working_hour','is_peak_hour'
]

# 2. Aplica la función
exog_con_interacciones = crear_interacciones_exogenas(exog, poly_cols)

# 3. Verifica el resultado
print(exog_con_interacciones.shape)
print([c for c in exog_con_interacciones.columns if c.startswith('poly__')][:10])

In [ ]:
from collections import OrderedDict
from lightgbm import LGBMRegressor
from skforecast.model_selection import backtesting_forecaster

# ✅ 1. Limpiar columnas exógenas (eliminar duplicados y mantener el orden)
exog_features = list(OrderedDict.fromkeys(exog_features))

# ✅ 2. Verificar que todas existan en exog_con_interacciones
exog_features = [col for col in exog_features if col in exog_con_interacciones.columns]

# ✅ 3. Construir DataFrame unificado (merge por índice)
data = df[['OFF Calls']].merge(
    exog_con_interacciones[exog_features],
    left_index=True,
    right_index=True,
    how='inner').astype('float32')

In [ ]:
# ✅ 4. Definir fechas de corte (ajusta si usas variables distintas)
end_train = '2024-04-30 23:59:00'
end_validation = '2025-03-31 23:59:00'

# ✅ 5. Dividir en train / val / test
data_train = data.loc[:end_train, :].copy()
data_val   = data.loc[end_train:end_validation, :].copy()
data_test  = data.loc[end_validation:, :].copy()

In [ ]:
data.head()

In [ ]:
# Redefinir el forecaster si no se ha hecho con las nuevas series
window_features = RollingFeatures(stats=["mean"], window_sizes=48*3)

forecaster = ForecasterRecursive(
                 regressor       = LGBMRegressor(random_state=15926, verbose=-1),
                 lags            = 48,
                 window_features = window_features
             )

# Entrenamiento usando hasta end_validation (train + val)
forecaster.fit(
    y    = data.loc[:end_validation, 'OFF Calls'],
    exog = data.loc[:end_validation, exog_features]

)

In [ ]:
# Configurar folds temporales
cv = TimeSeriesFold(
    steps              = 48,  # horizonte de predicción
    initial_train_size = len(data.loc[:end_validation]),
    refit              = False
)

# Ejecutar backtesting
metric, predictions = backtesting_forecaster(
    forecaster = forecaster,
    y          = data['OFF Calls'],
    exog       = data[exog_features],
    cv         = cv,
    metric     = 'mean_absolute_error',
    verbose    = True
)

# Visualizar resultados
display(metric)

In [ ]:
predictions.head(80000)

In [ ]:
import plotly.graph_objs as go

# Asegúrate de que las predicciones estén alineadas con el índice
fig = go.Figure()

# Trazar valores reales
fig.add_trace(go.Scatter(
    x = data_test.index,
    y = data_test['OFF Calls'],
    mode = 'lines',
    name = 'Valor real'
))

# Trazar predicciones (el índice de `predictions` ya está alineado)
fig.add_trace(go.Scatter(
    x = predictions.index,
    y = predictions['pred'],
    mode = 'lines',
    name = 'Predicción',
    line = dict(color='firebrick')
))

# Configurar layout
fig.update_layout(
    title = '📊 Predicción vs Real — OFF Calls',
    xaxis_title = 'Fecha y hora',
    yaxis_title = 'Número de llamadas',
    width = 900,
    height = 450,
    legend = dict(orientation="h", y=1.02, x=0),
    margin = dict(l=40, r=40, t=40, b=40)
)

fig.show()

In [ ]:
# Hyperparameters search
# ==============================================================================
forecaster = ForecasterRecursive(
                 regressor       = LGBMRegressor(random_state=15926, verbose=-1),
                 lags            = 48,  # This value will be replaced in the grid search
                 window_features = window_features
             )

# Lags used as predictors
lags_grid = [48, (1, 2, 3, 4, 49, 48, 5, 50, 34)]

# Regressor hyperparameters search space
def search_space(trial):
    search_space  = {
        'n_estimators' : trial.suggest_int('n_estimators', 300, 1000, step=100),
        'max_depth'    : trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
        'reg_alpha'    : trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda'   : trial.suggest_float('reg_lambda', 0, 1),
        'lags'         : trial.suggest_categorical('lags', lags_grid)
    }
    return search_space

# Folds training and validation
cv_search = TimeSeriesFold(
                steps              = 48,
                initial_train_size = len(data[:end_train]),
                refit              = False,
            )

results_search, frozen_trial = bayesian_search_forecaster(
                                   forecaster   = forecaster,
                                   y            = data.loc[:end_validation, 'OFF Calls'],
                                   exog         = data.loc[:end_validation, exog_features],
                                   cv           = cv_search,
                                   metric       = 'mean_absolute_error',
                                   search_space = search_space,
                                   n_trials     = 10,  # Increase for more exhaustive search
                                   return_best  = True
                               )

In [ ]:
# Search results
# ==============================================================================
best_params = results_search.at[0, 'params']
best_params = best_params | {'random_state': 15926, 'verbose': -1}
best_lags = results_search.at[0, 'lags']

In [ ]:
results_search.head(20)

In [ ]:
# Best model
# ==============================================================================
forecaster

In [ ]:
# Backtest final model on test data
# ==============================================================================
metric, predictions = backtesting_forecaster(
                          forecaster = forecaster,
                          y          = data['OFF Calls'],
                          exog       = data[exog_features],
                          cv         = cv,
                          metric     = 'mean_absolute_error'
                      )
display(metric)
predictions.head()

In [ ]:
import plotly.graph_objs as go

# Asegúrate de que las predicciones estén alineadas con el índice
fig = go.Figure()

# Trazar valores reales
fig.add_trace(go.Scatter(
    x = data_test.index,
    y = data_test['OFF Calls'],
    mode = 'lines',
    name = 'Valor real'
))

# Trazar predicciones (el índice de `predictions` ya está alineado)
fig.add_trace(go.Scatter(
    x = predictions.index,
    y = predictions['pred'],
    mode = 'lines',
    name = 'Predicción',
    line = dict(color='firebrick')
))

# Configurar layout
fig.update_layout(
    title = '📊 Predicción vs Real — OFF Calls',
    xaxis_title = 'Fecha y hora',
    yaxis_title = 'Número de llamadas',
    width = 900,
    height = 450,
    legend = dict(orientation="h", y=1.02, x=0),
    margin = dict(l=40, r=40, t=40, b=40)
)

fig.show()

### Selección de caracteristicas



In [ ]:
# Create forecaster
# ==============================================================================
regressor = LGBMRegressor(
                n_estimators = 100,
                max_depth    = 4,
                random_state = 15926,
                verbose      = -1
            )

forecaster = ForecasterRecursive(
                 regressor       = regressor,
                 lags            = best_lags,
                 window_features = window_features
             )

# Recursive feature elimination with cross-validation
# ==============================================================================
warnings.filterwarnings("ignore", message="X does not have valid feature names.*")
selector = RFECV(
    estimator = regressor,
    step      = 1,
    cv        = 3,
)
lags_select, window_features_select, exog_select = select_features(
    forecaster      = forecaster,
    selector        = selector,
    y               = data_train['OFF Calls'],
    exog            = data_train[exog_features],
    select_only     = None,
    force_inclusion = None,
    subsample       = 0.5,  # Subsample to speed up the process
    random_state    = 123,
    verbose         = True,
)

In [ ]:
# Create a forecaster with the selected features
# ==============================================================================
forecaster = ForecasterRecursive(
                regressor       = LGBMRegressor(**best_params),
                lags            = lags_select,
                window_features = window_features
             )
# Backtesting model with exogenous variables on test data
# ==============================================================================
metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = data['OFF Calls'],
                            exog       = data[exog_select],
                            cv         = cv,
                            metric     = 'mean_absolute_error'
                      )
metric

### Pronostico Probabilistico: Prediccion de los intervalos



In [ ]:
# Create and train forecaster
# ==============================================================================
forecaster = ForecasterRecursive(
                 regressor       = LGBMRegressor(**best_params),
                 lags            = lags_select,
                 window_features = window_features,
                 binner_kwargs   = {"n_bins": 5}
             )
forecaster.fit(
    y    = data.loc[:end_train, 'OFF Calls'],
    exog = data.loc[:end_train, exog_select],
    store_in_sample_residuals = True
)

In [ ]:
# Predict intervals
# ==============================================================================
# Since the model has been trained with exogenous variables, they must be provided
# for the prediction.
predictions = forecaster.predict_interval(
                  exog     = data.loc[end_train:, exog_select],
                  steps    = 48,
                  interval = [5, 95],
                  method  = 'conformal',
              )
predictions.head()

In [ ]:
# Backtesting on validation data to obtain out-sample residuals
# ==============================================================================
cv = TimeSeriesFold(
        steps              = 48,
        initial_train_size = len(data.loc[:end_train]),
        refit              = False,
)
_, predictions_val = backtesting_forecaster(
                         forecaster = forecaster,
                         y          = data.loc[:end_validation, 'OFF Calls'],
                         exog       = data.loc[:end_validation, exog_select],
                         cv         = cv,
                         metric     = 'mean_absolute_error'
                     )

In [ ]:
# Out-sample residuals distribution
# ==============================================================================
residuals = data.loc[predictions_val.index, 'OFF Calls'] - predictions_val['pred']
print(pd.Series(np.where(residuals < 0, 'negative', 'positive')).value_counts())
plt.rcParams.update({'font.size': 8})
_ = plot_residuals(residuals=residuals, figsize=(7, 4))

In [ ]:
# Store out-sample residuals in the forecaster
# ==============================================================================
forecaster.set_out_sample_residuals(
    y_true = data.loc[predictions_val.index, 'OFF Calls'],
    y_pred = predictions_val['pred']
)

In [ ]:
# Backtesting with prediction intervals in test data using out-sample residuals
# ==============================================================================
cv = TimeSeriesFold(
        steps              = 48,
        initial_train_size = len(data.loc[:end_validation]),
        refit              = False,
)
metric, predictions = backtesting_forecaster(
                            forecaster              = forecaster,
                            y                       = data['OFF Calls'],
                            exog                    = data[exog_select],
                            cv                      = cv,
                            metric                  = 'mean_absolute_error',
                            interval                = [5, 95],
                            interval_method         = 'conformal',
                            use_in_sample_residuals = False,  # out-sample residuals
                            use_binned_residuals    = True,   # Intervals conditioned to the predicted values
                       )
predictions.head(5)

In [ ]:
# Plot prediction intervals vs real value
# ==============================================================================
fig = go.Figure([
    go.Scatter(
        name='Prediction', x=predictions.index, y=predictions['pred'], mode='lines',
    ),
    go.Scatter(
        name='Real value', x=data_test.index, y=data_test['OFF Calls'], mode='lines',
    ),
    go.Scatter(
        name='Upper Bound', x=predictions.index, y=predictions['upper_bound'],
        mode='lines', marker=dict(color="#444"), line=dict(width=0), showlegend=False
    ),
    go.Scatter(
        name='Lower Bound', x=predictions.index, y=predictions['lower_bound'],
        marker=dict(color="#444"), line=dict(width=0), mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)', fill='tonexty', showlegend=False
    )
])
fig.update_layout(
    title="Real value vs predicted in test data",
    xaxis_title="Date time",
    yaxis_title="OFF Calls",
    width=800,
    height=400,
    margin=dict(l=20, r=20, t=35, b=20),
    hovermode="x",
    legend=dict(orientation="h", yanchor="top", y=1.1, xanchor="left", x=0.001)
)
fig.show()

In [ ]:
# Predicted interval coverage (on test data)
# ==============================================================================
coverage = calculate_coverage(
              y_true       = data.loc[end_validation:, "OFF Calls"],
              lower_bound  = predictions["lower_bound"],
              upper_bound  = predictions["upper_bound"]
           )
area = (predictions['upper_bound'] - predictions['lower_bound']).sum()
print(f"Total area of the interval: {round(area, 2)}")
print(f"Predicted interval coverage: {round(100 * coverage, 2)} %")

#### Explicabilidad e interpretabilidad del modelo



In [ ]:
# Create and train forecaster
# ==============================================================================
forecaster = ForecasterRecursive(
                 regressor       = LGBMRegressor(**best_params),
                 lags            = lags_select,
                 window_features = window_features
             )
forecaster.fit(
    y    = data.loc[:end_validation, 'OFF Calls'],
    exog = data.loc[:end_validation, exog_select]
)


In [ ]:
# Model-specific feature importances
# ==============================================================================
feature_importances = forecaster.get_feature_importances()
feature_importances.head(10)

#### Shap Values



In [ ]:
# Training matrices used by the forecaster to fit the internal regressor
# ==============================================================================
X_train, y_train = forecaster.create_train_X_y(
                       y    = data_train['OFF Calls'],
                       exog = data_train[exog_select]
                   )
display(X_train.head(3))
display(y_train.head(3))

In [ ]:
# Create SHAP explainer (for three base models)
# ==============================================================================
shap.initjs()
explainer = shap.TreeExplainer(forecaster.regressor)

# Sample 50% of the data to speed up the calculation
rng = np.random.default_rng(seed=785412)
sample = rng.choice(X_train.index, size=int(len(X_train)*0.5), replace=False)
X_train_sample = X_train.loc[sample, :]
shap_values = explainer.shap_values(X_train_sample)

In [ ]:
# Shap summary plot (top 10)
# ==============================================================================
shap.summary_plot(shap_values, X_train_sample, max_display=10, show=False)
fig, ax = plt.gcf(), plt.gca()
ax.set_title("SHAP Summary plot")
ax.tick_params(labelsize=8)
fig.set_size_inches(6, 4.5)

In [ ]:
# Backtesting returning the predictors
# ==============================================================================
cv = TimeSeriesFold(
        steps              = 48,
        initial_train_size = len(data.loc[:end_validation]),
)
_, predictions = backtesting_forecaster(
                        forecaster        = forecaster,
                        y                 = data['OFF Calls'],
                        exog              = data[exog_select],
                        cv                = cv,
                        metric            = 'mean_absolute_error',
                        return_predictors = True,
                )

In [ ]:
predictions.head(8000)

In [ ]:
# Waterfall for a single prediction generated during backtesting
# ==============================================================================
predictions = predictions.astype(data[exog_select].dtypes) # Ensure that the types are the same
iloc_predicted_date = predictions.index.get_loc('2025-04-01 21:30:00')
shap_values_single = explainer(predictions.iloc[:, 2:])
shap.plots.waterfall(shap_values_single[iloc_predicted_date], show=False)
fig, ax = plt.gcf(), plt.gca()
fig.set_size_inches(8, 3.5)
ax_list = fig.axes
ax = ax_list[0]
ax.tick_params(labelsize=8)
ax.set
plt.show()

In [ ]:
# Forceplot for a single prediction generated during backtesting
# ==============================================================================
shap.force_plot(
    base_value  = shap_values_single.base_values[iloc_predicted_date],
    shap_values = shap_values_single.values[iloc_predicted_date],
    features    = predictions.iloc[iloc_predicted_date, 2:],
)

TSF=sla,(target tsg,20 goal de aba,offered forecast*2,forecast aht)*100)



```

```



Service Level=  \(Answered calls in threshold\)/\(Agent Offered Calls\)

Ejemplo de una variación en la fórmula de SL

Service Level \(2\)=  \(Answered calls in threshold\-abn&lt;10 sec\)/\(Agent Offered Calls\)

ASA=\(Handled Time\)/\(Agent handled Calls\)

Abandon=\(Abandon Calls\)/\(Agent Offered Calls\)

Occupancy=1\-Avail%

AHT=\(BCW\+TT\+HT\+ACW\)/\(Agent Handled Calls\)

Calls per hour \(Agent Handled Calls\)/\(Staff Hours\)

Calls per FTE \(Agent Handled Calls\)/\(Total FTE\)

Sells per hour \(Total Sells\)/\(Staff Hours\)

Ratio Calls/Sells \(Handled Calls\)/Sells

Offered,aht=forecast

goals: TSF: Es el porcentaje de llamadas atendidas dentro de un umbral especifico de tiempo

TSF Goal \(75/20\)

75% de las llamadas deben ser atendidas antes de los 20 segundos de espera.

TSF = llamadas en Umbral /llamadas ofrecidas

AHT: Talk Time \+ hold time \+ after call work / numbero de conversasiones de los clientes

ABA: Es el porcentaje de llamadas finalizadas por parte del cliente antes de ser atendidas por un CCA

ABA Goal: 6%

El porcentaje de llamadas abandonadas debe de ser 6%    Llamadas abandonadas / llamadas ofreciadas

ASA  es el tiempo promedi oque dura una llamada en la linea de espera antes de ser aatendida por un CCA

ASA Goal. 35s donde el tiempo rpomedio de respuesta debe ser hasta 35 segundos

AS = Tiempo total de espera / llamadas atendidas



SCR: Identificación de picos: 9:00 am hasta la 1 de la tarde. las horas madrugada son horas con poca información

ca: 9 am hasta la 1 de la tarde. En la tarde es de 5 a 7 de la noche.

ca: 450 s

scr: 520 s

aht aumenta los picos cuando offerd aumentado, caudas de aplicativo. problemas de cola horas valle.

TSF: %0 ABA 0 segundos ASA.
